In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import linregress
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.externals import joblib
from sklearn.naive_bayes import GaussianNB

from keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, concatenate, Conv2D
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
def load_5models_from_disk():
    models = []
    for i in range(5):
        json_file = open("./model" + str(i) +".json", 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
        # load weights into new model
        loaded_model.load_weights("temp" + str(i) +".hdf5")
        print("Loaded model from disk")

        # evaluate loaded model on test data
        loaded_model.compile(optimizer='rmsprop',
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])
        models.append(loaded_model)
    return models

In [3]:
def shrink_array(array,size):
    
    ratio = float(len(array)) / float(size+1)
    res = []
    for i in range(size):
        res.append(np.mean(array[math.floor(i*ratio):math.ceil((i+2.0)*ratio)], axis = 0))
    return np.array(res)

In [4]:
train_data = pd.DataFrame.from_csv("../../data/database/train_data.csv")
test_data = pd.DataFrame.from_csv("../../data/database/test_data.csv")
f_df = pd.DataFrame.from_csv("../../data/gesture_feature_df.csv")
train_data.columns

Index(['unix_timestamp', 'TagName', 'tester_id', 'v_1', 'v_2', 'v_3', 'd_1',
       'd_2', 'd_3', 'global_acc1', 'global_acc2', 'global_acc3',
       'v_12_square', 'acc_12_square', 'd_12_square'],
      dtype='object')

In [5]:
feature_cols = ['global_acc3','acc_12_square']

In [6]:
def get_feature_label(data, N):
    groups = data.groupby(['TagName','tester_id'])
    keys = groups.groups.keys()
    y = []
    X = []
    for k in keys:
        frame_feature = shrink_array(groups.get_group(k)[feature_cols].values, N)
        X.append(frame_feature)
        y.append(k[0])
    return np.array(X),np.array(y)

def get_feature_label_f(data, N):
    groups = data.groupby(['TagName','tester_id'])
    keys = groups.groups.keys()
    y = []
    X = []
    f = []
    for k in keys:
        frame_feature = shrink_array(groups.get_group(k)[feature_cols].values, N)
        X.append(frame_feature)
        y.append(k[0])
        f.append(f_df[(f_df.TagName == k[0]) & (f_df.tester_id == k[1])].d_change.values[0])
    return np.array(X),np.array(y),np.array(f)

In [7]:
def SVC_training(X_train, y_train,x_test, y_test, f_test):
    if len(X_train.shape) > 2:
        X_train = X_train.reshape(list(X_train.shape)[0],-1)
        x_test = x_test.reshape(list(x_test.shape)[0],-1)
    overal_max = []
    for k in ['poly','rbf','linear']:
        max_score = 0
        max_i = 0
        max_res = []
        for i in range(1,4):
            i = i/2
            clf4 = SVC(kernel=k, C=i, degree=3, verbose = False)

            clf4.fit(X_train, y_train) 
            # joblib.dump(clf4, '../../Results/baseline SVC 0.80 raw data acc with gyro 200 chunk.pkl') 
            res = clf4.predict(x_test)
            score = accuracy_score(y_test, res)
            if score>max_score:
                max_score = score
                max_i = i
                max_res = res
        print(k)
        print("max score: " + str(max_score) + " C = " + str(max_i))
        
        print(classification_report(y_test, max_res))
        print(confusion_matrix(y_test, max_res))
        if ensemble_06 is True:
            for i in range(len(max_res)):
                if (max_res[i] =='Tag0') or (max_res[i] == "Tag6"):
                    max_res[i] = rf_clf.predict(f_test[i])[0]
            print("-----ensembled---------")
            print(classification_report(y_test, max_res))
            print(confusion_matrix(y_test, max_res))
        overal_max.append(max_score)
    return overal_max
            
        
def RF_training(X_train, y_train,x_test, y_test, f_test):
    if len(X_train.shape) > 2:
        X_train = X_train.reshape(list(X_train.shape)[0],-1)
        x_test = x_test.reshape(list(x_test.shape)[0],-1)

        clf4 = RandomForestClassifier(n_estimators=30)

        clf4.fit(X_train, y_train) 
        # joblib.dump(clf4, '../../Results/baseline SVC 0.80 raw data acc with gyro 200 chunk.pkl') 
        res = clf4.predict(x_test)
        score = accuracy_score(y_test, res)
        
        print(classification_report(y_test, res))
        print(confusion_matrix(y_test, res))
        if ensemble_06 is True:
            for i in range(len(res)):
                if (res[i] =='Tag0') or (res[i] == "Tag6"):
                    res[i] = rf_clf.predict(f_test[i])[0]
            print("-----ensembled---------")
            print(classification_report(y_test, res))
            print(confusion_matrix(y_test, res))
        return(accuracy_score(y_test, res))

    
def ada_training(X_train, y_train,x_test, y_test, f_test):
    if len(X_train.shape) > 2:
        X_train = X_train.reshape(list(X_train.shape)[0],-1)
        x_test = x_test.reshape(list(x_test.shape)[0],-1)

        clf4 = AdaBoostClassifier(n_estimators=30)

        clf4.fit(X_train, y_train) 
        # joblib.dump(clf4, '../../Results/baseline SVC 0.80 raw data acc with gyro 200 chunk.pkl') 
        res = clf4.predict(x_test)
        score = accuracy_score(y_test, res)
        
        print(classification_report(y_test, res))
        print(confusion_matrix(y_test, res))
        if ensemble_06 is True:
            for i in range(len(res)):
                if (res[i] =='Tag0') or (res[i] == "Tag6"):
                    res[i] = rf_clf.predict(f_test[i])[0]
            print("-----ensembled---------")
            print(classification_report(y_test, res))
            print(confusion_matrix(y_test, res))
        return(accuracy_score(y_test, res))
    
def nb_training(X_train, y_train,x_test, y_test, f_test):
    if len(X_train.shape) > 2:
        X_train = X_train.reshape(list(X_train.shape)[0],-1)
        x_test = x_test.reshape(list(x_test.shape)[0],-1)

        clf4 = GaussianNB()

        clf4.fit(X_train, y_train) 
        # joblib.dump(clf4, '../../Results/baseline SVC 0.80 raw data acc with gyro 200 chunk.pkl') 
        res = clf4.predict(x_test)
        score = accuracy_score(y_test, res)
        
        print(classification_report(y_test, res))
        print(confusion_matrix(y_test, res))
        if ensemble_06 is True:
            for i in range(len(res)):
                if (res[i] =='Tag0') or (res[i] == "Tag6"):
                    res[i] = rf_clf.predict(f_test[i])[0]
            print("-----ensembled---------")
            print(classification_report(y_test, res))
            print(confusion_matrix(y_test, res))
        return(accuracy_score(y_test, res))


def DL_training(X_train, y_train,x_test, y_test, f_test):

    y = np.concatenate([y_train,y_test])
    tag_list = []
    for i in range(10):
        tag_list.append(['Tag'+str(i),i])
    for i in tag_list:
        tag_str = i[0]
        tag_int = i[1]
        y[y==tag_str] = tag_int
    y_categorical = to_categorical(y)
    
    y_train_cate = y_categorical[:len(y_train)]
    y_test_cate = y_categorical[len(y_train):]

    X_train = X_train.reshape(list(X_train.shape)[0],-1)
    x_test = x_test.reshape(list(x_test.shape)[0],-1)
    
    for i in range(5):
        # This returns a tensor
        inputs = Input(shape=(X_train.shape[1:]))

        # a layer instance is callable on a tensor, and returns a tensor
        layer1 = Dense(64, activation='relu')(inputs)
        layer2 = Dense(128, activation='relu')(layer1)
        layer3 = Dense(64, activation='relu')(layer2)
        layer4 = Dense(32, activation='relu')(layer3)
        predictions = Dense(10, activation='softmax')(layer4)

        mcp = ModelCheckpoint("./temp" + str(i) + ".hdf5", monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
        model = Model(inputs=inputs, outputs=predictions)
#         print(model.summary())
        model.compile(optimizer='rmsprop',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        model_his = model.fit(X_train, y_train_cate, batch_size=32, epochs=40, verbose = 0,
                              validation_data=(x_test, y_test_cate), callbacks = [mcp]
                             )  # starts training

        model_json = model.to_json()
        with open("./model" + str(i) +".json", "w") as json_file:
            json_file.write(model_json)
            
    models = load_5models_from_disk()
    scores = []
    tests = []
    predicts = []
    f = []
    for model in models:
        res = model.predict(x_test)
        scores.append(accuracy_score(np.argmax(y_test_cate,1), np.argmax(res, 1)))
        tests += np.argmax(y_test_cate,1).tolist()
        f += f_test.tolist()
        predicts += np.argmax(res, 1).tolist()
    
    print(classification_report(tests, predicts))
    print(confusion_matrix(tests, predicts))
    if ensemble_06 is True:
        for i in range(len(predicts)):
            if (predicts[i] ==0) or (predicts[i] == 6):
                if rf_clf.predict(f[i])[0] == 'Tag0':
                    predicts[i] = 0
                else:
                    predicts[i] = 6
        print("-----ensembled---------")
        print(classification_report(tests, predicts))
        print(confusion_matrix(tests, predicts))
    return accuracy_score(tests, predicts)
    
def CONV1d_training(X_train, y_train,x_test, y_test, f_test):
    y = np.concatenate([y_train,y_test])
    tag_list = []
    for i in range(10):
        tag_list.append(['Tag'+str(i),i])
    for i in tag_list:
        tag_str = i[0]
        tag_int = i[1]
        y[y==tag_str] = tag_int
    y_categorical = to_categorical(y)
    
    y_train_cate = y_categorical[:len(y_train)]
    y_test_cate = y_categorical[len(y_train):]

    
    for i in range(5):
        input_val1 = Input(shape=X_train.shape[1:])

        con1 = Conv1D(filters=30,kernel_size=5)(input_val1)
        max_pooling_1d_1 = MaxPooling1D(pool_size=2, strides=None, padding='valid')(con1)
        flat_1 = Flatten()(max_pooling_1d_1)
        layer2 = Dense(128, activation='relu')(flat_1)
        layer4 = Dense(32, activation='relu')(layer2)
        predictions = Dense(y_categorical.shape[-1], activation='softmax')(layer4)

        model = Model(inputs = input_val1, outputs=predictions)
#         print(model.summary())
        mcp = ModelCheckpoint("./temp" + str(i) + ".hdf5", monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    
        model.compile(optimizer='rmsprop',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
    
        model_his = model.fit(X_train, y_train_cate, batch_size=32, epochs=40, verbose = 0,
                              validation_data=(x_test, y_test_cate), callbacks = [mcp]
                             )  # starts training

        model_json = model.to_json()
        with open("./model" + str(i) +".json", "w") as json_file:
            json_file.write(model_json)
            
    models = load_5models_from_disk()
    scores = []
    tests = []
    predicts = []
    f = []
    
    for model in models:
        res = model.predict(x_test)
        scores.append(accuracy_score(np.argmax(y_test_cate,1), np.argmax(res, 1)))
        tests += np.argmax(y_test_cate,1).tolist()
        predicts += np.argmax(res, 1).tolist()
        f += f_test.tolist()
        
    print(classification_report(tests, predicts))
    print(confusion_matrix(tests, predicts))
    if ensemble_06 is True:
        for i in range(len(predicts)):
            if (predicts[i] ==0) or (predicts[i] == 6):
                if rf_clf.predict(f[i])[0] == 'Tag0':
                    predicts[i] = 0
                else:
                    predicts[i] = 6
        print("-----ensembled---------")
        print(classification_report(tests, predicts))
        print(confusion_matrix(tests, predicts))  
    return accuracy_score(tests, predicts)

In [8]:
ensemble_06 = True
if ensemble_06 is True:
    rf_clf = joblib.load('./binary_model.pkl') 

In [9]:
rf = []
svc = []
dl = []
conv = []
for N in range(6,27,2):
    X_train, y_train = get_feature_label(train_data, N)
    x_test, y_test, f_test = get_feature_label_f(test_data, N)
#     print(X_train.shape)
#     print(y_train.shape)
    rf.append(RF_training(X_train, y_train, x_test, y_test, f_test))
    svc.append(SVC_training(X_train, y_train, x_test, y_test, f_test))
    dl.append(DL_training(X_train, y_train, x_test, y_test, f_test))
    conv.append(CONV1d_training(X_train, y_train, x_test, y_test, f_test))

             precision    recall  f1-score   support

       Tag0       0.63      0.57      0.60        21
       Tag1       1.00      0.90      0.95        21
       Tag2       0.81      0.81      0.81        21
       Tag3       0.69      0.86      0.77        21
       Tag4       0.69      0.86      0.77        21
       Tag5       0.71      0.48      0.57        21
       Tag6       0.62      0.62      0.62        21
       Tag7       0.69      0.86      0.77        21
       Tag8       0.83      0.71      0.77        21
       Tag9       0.70      0.67      0.68        21

avg / total       0.74      0.73      0.73       210

[[12  0  0  0  2  0  5  1  1  0]
 [ 0 19  0  0  0  0  0  2  0  0]
 [ 1  0 17  3  0  0  0  0  0  0]
 [ 0  0  2 18  0  0  0  1  0  0]
 [ 0  0  0  1 18  0  0  0  0  2]
 [ 0  0  0  1  2 10  2  3  1  2]
 [ 6  0  0  0  1  0 13  0  1  0]
 [ 0  0  1  1  0  0  0 18  0  1]
 [ 0  0  1  0  0  4  0  0 15  1]
 [ 0  0  0  2  3  0  1  1  0 14]]
-----ensembled---------
      

-----ensembled---------
             precision    recall  f1-score   support

          0       0.84      0.93      0.89       105
          1       1.00      0.95      0.98       105
          2       0.92      0.93      0.93       105
          3       0.85      0.96      0.90       105
          4       0.82      0.88      0.85       105
          5       0.81      0.69      0.74       105
          6       0.89      0.81      0.85       105
          7       0.87      0.95      0.91       105
          8       0.83      0.82      0.83       105
          9       0.82      0.74      0.78       105

avg / total       0.87      0.87      0.86      1050

[[ 98   0   0   0   0   0   5   0   0   2]
 [  0 100   0   0   0   0   0   5   0   0]
 [  0   0  98   5   0   0   0   0   1   1]
 [  0   0   1 101   0   0   0   0   0   3]
 [  0   0   0   5  92   0   0   0   0   8]
 [  6   0   4   5   1  72   5   2   7   3]
 [ 10   0   0   0   0   0  85   1   9   0]
 [  0   0   3   2   0   0   0 100   

linear
max score: 0.819047619048 C = 1.0
             precision    recall  f1-score   support

       Tag0       0.69      0.86      0.77        21
       Tag1       0.91      0.95      0.93        21
       Tag2       0.90      0.90      0.90        21
       Tag3       0.72      1.00      0.84        21
       Tag4       0.90      0.86      0.88        21
       Tag5       0.75      0.57      0.65        21
       Tag6       0.80      0.76      0.78        21
       Tag7       0.81      0.81      0.81        21
       Tag8       0.94      0.71      0.81        21
       Tag9       0.84      0.76      0.80        21

avg / total       0.83      0.82      0.82       210

[[18  0  0  0  0  1  1  0  1  0]
 [ 0 20  0  0  0  0  0  1  0  0]
 [ 0  0 19  2  0  0  0  0  0  0]
 [ 0  0  0 21  0  0  0  0  0  0]
 [ 0  0  0  1 18  0  0  1  0  1]
 [ 2  0  0  3  1 12  1  0  0  2]
 [ 5  0  0  0  0  0 16  0  0  0]
 [ 0  2  2  0  0  0  0 17  0  0]
 [ 1  0  0  0  0  3  2  0 15  0]
 [ 0  0  0  2  1  0  0 

poly
max score: 0.871428571429 C = 0.5
             precision    recall  f1-score   support

       Tag0       0.80      0.95      0.87        21
       Tag1       0.91      0.95      0.93        21
       Tag2       0.94      0.71      0.81        21
       Tag3       0.81      1.00      0.89        21
       Tag4       0.95      0.95      0.95        21
       Tag5       0.76      0.76      0.76        21
       Tag6       0.89      0.76      0.82        21
       Tag7       0.86      0.86      0.86        21
       Tag8       0.90      0.86      0.88        21
       Tag9       0.95      0.90      0.93        21

avg / total       0.88      0.87      0.87       210

[[20  0  0  0  0  0  1  0  0  0]
 [ 0 20  0  0  0  0  0  1  0  0]
 [ 0  1 15  2  0  2  1  0  0  0]
 [ 0  0  0 21  0  0  0  0  0  0]
 [ 0  0  0  0 20  0  0  0  0  1]
 [ 1  0  0  1  0 16  0  1  2  0]
 [ 4  0  0  1  0  0 16  0  0  0]
 [ 0  1  1  1  0  0  0 18  0  0]
 [ 0  0  0  0  0  3  0  0 18  0]
 [ 0  0  0  0  1  0  0  1

Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
             precision    recall  f1-score   support

          0       0.82      0.89      0.85       105
          1       0.99      0.95      0.97       105
          2       0.96      0.93      0.95       105
          3       0.90      0.99      0.95       105
          4       0.99      0.95      0.97       105
          5       0.91      0.79      0.85       105
          6       0.77      0.82      0.80       105
          7       0.96      0.95      0.96       105
          8       0.89      0.88      0.88       105
          9       0.93      0.96      0.94       105

avg / total       0.91      0.91      0.91      1050

[[ 93   0   0   0   0   0  12   0   0   0]
 [  0 100   0   0   0   0   0   1   0   4]
 [  0   0  98   5   0   1   0   0   1   0]
 [  0   0   1 104   0   0   0   0   0   0]
 [  1   0   0   0 100   0   0   0   0   4]
 [  0   0   2   5   0  83   5  

...

poly
max score: 0.852380952381 C = 0.5
             precision    recall  f1-score   support

       Tag0       0.78      0.86      0.82        21
       Tag1       1.00      0.86      0.92        21
       Tag2       0.86      0.90      0.88        21
       Tag3       0.84      1.00      0.91        21
       Tag4       0.95      0.90      0.93        21
       Tag5       0.74      0.81      0.77        21
       Tag6       0.85      0.81      0.83        21
       Tag7       0.86      0.90      0.88        21
       Tag8       0.84      0.76      0.80        21
       Tag9       0.83      0.71      0.77        21

avg / total       0.86      0.85      0.85       210

[[18  0  0  0  0  0  2  0  0  1]
 [ 0 18  1  0  0  1  0  0  0  1]
 [ 0  0 19  1  0  0  0  0  1  0]
 [ 0  0  0 21  0  0  0  0  0  0]
 [ 0  0  0  1 19  0  0  0  0  1]
 [ 0  0  0  1  0 17  1  0  2  0]
 [ 4  0  0  0  0  0 17  0  0  0]
 [ 0  0  1  1  0  0  0 19  0  0]
 [ 0  0  1  0  0  4  0  0 16  0]
 [ 1  0  0  0  1  1  0  3

Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
             precision    recall  f1-score   support

          0       0.73      0.92      0.82       105
          1       0.97      0.94      0.96       105
          2       1.00      0.95      0.98       105
          3       0.91      1.00      0.95       105
          4       0.96      0.95      0.96       105
          5       0.88      0.84      0.86       105
          6       0.81      0.70      0.76       105
          7       0.97      0.95      0.96       105
          8       0.93      0.86      0.89       105
          9       0.92      0.92      0.92       105

avg / total       0.91      0.90      0.90      1050

[[ 97   0   0   0   0   0   8   0   0   0]
 [  0  99   0   0   1   0   0   0   0   5]
 [  0   0 100   5   0   0   0   0   0   0]
 [  0   0   0 105   0   0   0   0   0   0]
 [  2   0   0   0 100   0   0   0   0   3]
 [  2   0   0   5   0  88   3  

linear
max score: 0.77619047619 C = 1.0
             precision    recall  f1-score   support

       Tag0       0.60      0.86      0.71        21
       Tag1       0.91      0.95      0.93        21
       Tag2       0.86      0.86      0.86        21
       Tag3       0.68      0.90      0.78        21
       Tag4       0.94      0.81      0.87        21
       Tag5       0.76      0.62      0.68        21
       Tag6       0.80      0.76      0.78        21
       Tag7       0.70      0.76      0.73        21
       Tag8       1.00      0.67      0.80        21
       Tag9       0.71      0.57      0.63        21

avg / total       0.80      0.78      0.78       210

[[18  0  0  0  0  0  2  0  0  1]
 [ 0 20  0  0  0  0  0  0  0  1]
 [ 0  0 18  3  0  0  0  0  0  0]
 [ 0  0  2 19  0  0  0  0  0  0]
 [ 1  0  0  0 17  0  0  1  0  2]
 [ 2  0  0  4  0 13  1  0  0  1]
 [ 5  0  0  0  0  0 16  0  0  0]
 [ 0  2  1  1  0  1  0 16  0  0]
 [ 4  0  0  0  0  2  1  0 14  0]
 [ 0  0  0  1  1  1  0  

poly
max score: 0.833333333333 C = 0.5
             precision    recall  f1-score   support

       Tag0       0.74      0.81      0.77        21
       Tag1       1.00      0.86      0.92        21
       Tag2       0.91      0.95      0.93        21
       Tag3       0.80      0.95      0.87        21
       Tag4       0.95      0.86      0.90        21
       Tag5       0.68      0.81      0.74        21
       Tag6       0.83      0.71      0.77        21
       Tag7       0.86      0.90      0.88        21
       Tag8       0.89      0.76      0.82        21
       Tag9       0.75      0.71      0.73        21

avg / total       0.84      0.83      0.83       210

[[17  0  0  0  0  1  2  0  0  1]
 [ 0 18  1  0  0  1  0  0  0  1]
 [ 0  0 20  1  0  0  0  0  0  0]
 [ 0  0  0 20  0  0  0  0  0  1]
 [ 0  0  0  1 18  0  0  0  0  2]
 [ 0  0  0  1  0 17  1  0  2  0]
 [ 5  0  0  1  0  0 15  0  0  0]
 [ 0  0  0  1  0  1  0 19  0  0]
 [ 0  0  1  0  0  4  0  0 16  0]
 [ 1  0  0  0  1  1  0  3

Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
             precision    recall  f1-score   support

          0       0.78      0.89      0.83       105
          1       1.00      0.94      0.97       105
          2       0.97      0.92      0.95       105
          3       0.89      0.98      0.93       105
          4       0.96      0.96      0.96       105
          5       0.92      0.85      0.88       105
          6       0.81      0.81      0.81       105
          7       0.98      0.96      0.97       105
          8       0.92      0.87      0.89       105
          9       0.92      0.93      0.92       105

avg / total       0.91      0.91      0.91      1050

[[ 93   0   0   0   0   0  11   0   0   1]
 [  0  99   0   0   0   0   0   0   0   6]
 [  0   0  97   6   0   0   0   0   2   0]
 [  0   0   2 103   0   0   0   0   0   0]
 [  2   0   0   0 101   0   0   0   0   2]
 [  2   0   0   5   1  89   2  

In [15]:
print(dl)

[0.8666666666666667, 0.89714285714285713, 0.91428571428571426, 0.91619047619047622, 0.92190476190476189, 0.92666666666666664, 0.92095238095238097, 0.92285714285714282, 0.92190476190476189, 0.92380952380952386, 0.91809523809523808]


In [16]:
print(conv)

[0.83904761904761904, 0.89714285714285713, 0.92571428571428571, 0.92190476190476189, 0.93333333333333335, 0.93047619047619046, 0.93238095238095242, 0.92380952380952386, 0.92761904761904757, 0.92761904761904757, 0.92666666666666664]


In [17]:
print(rf)

[0.77142857142857146, 0.8666666666666667, 0.85238095238095235, 0.86190476190476195, 0.81904761904761902, 0.85238095238095235, 0.83809523809523812, 0.82380952380952377, 0.82380952380952377, 0.81904761904761902, 0.81428571428571428]


In [18]:
print(svc)

[[0.85238095238095235, 0.77619047619047621, 0.79523809523809519], [0.86190476190476195, 0.82380952380952377, 0.81904761904761902], [0.87142857142857144, 0.84285714285714286, 0.81428571428571428], [0.87142857142857144, 0.8571428571428571, 0.80476190476190479], [0.87619047619047619, 0.83809523809523812, 0.80000000000000004], [0.86190476190476195, 0.82857142857142863, 0.81428571428571428], [0.86190476190476195, 0.81428571428571428, 0.80476190476190479], [0.85238095238095235, 0.79047619047619044, 0.77142857142857146], [0.85238095238095235, 0.79047619047619044, 0.7857142857142857], [0.84285714285714286, 0.76666666666666672, 0.77619047619047621], [0.83333333333333337, 0.74761904761904763, 0.80952380952380953]]


In [10]:
N = 14
X_train, y_train = get_feature_label(train_data, N)
x_test, y_test, f_test = get_feature_label_f(test_data, N)

CONV1d_training(X_train, y_train, x_test, y_test, f_test)

Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
             precision    recall  f1-score   support

          0       0.78      0.86      0.82       105
          1       0.99      0.95      0.97       105
          2       0.96      0.93      0.95       105
          3       0.91      1.00      0.95       105
          4       0.99      0.95      0.97       105
          5       0.90      0.85      0.87       105
          6       0.76      0.79      0.78       105
          7       0.99      0.96      0.98       105
          8       0.93      0.85      0.89       105
          9       0.93      0.97      0.95       105

avg / total       0.91      0.91      0.91      1050

[[ 90   0   0   0   0   0  15   0   0   0]
 [  0 100   0   0   0   0   0   0   0   5]
 [  0   0  98   6   0   0   0   0   1   0]
 [  0   0   0 105   0   0   0   0   0   0]
 [  3   0   0   0 100   0   0   0   0   2]
 [  1   0   2   3   0  89   4  

0.93142857142857138

In [14]:
open("converteds_model.tflite", "wb").write(models[1])

TypeError: a bytes-like object is required, not 'Model'

In [15]:
models[1]